<a href="https://colab.research.google.com/github/FCosmin27/ocs/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import random

In [12]:
upper_bound = 100
lower_bound = -100
dimension = 10

In [13]:
def check_x(x):
    for i in range(len(x)):
        if x[i] > upper_bound:
            x[i] = upper_bound
        elif x[i] < lower_bound:
            x[i] = lower_bound
    return x

In [14]:
x = np.random.normal(size=dimension)

x = check_x(x)
print(x)

[ 0.15632162  0.55495605 -2.34349392  0.79148215 -0.29618048  0.96514359
 -0.58420348  0.60262036  1.14198474  0.97772021]


In [15]:
def f1(x):
    return np.sum(np.square(x))

In [16]:
def f2(x):
    result = 0
    for i in range(len(x)):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result

In [17]:
def f3(x):
    result = 0
    for i in range(1, len(x)):
        result += 1e6 ** ((i-1)/(len(x)-1)) * x[i]**2

    return result

In [18]:
def f4(x):
    result = 0
    for i in range(len(x)):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result * (1 + 0.4 * np.abs(np.random.randn()))

In [19]:
def f5(x):
    A = np.random.randint(-500, 501, size=(len(x), len(x)))

    while np.linalg.det(A) == 0:
        A = np.random.randint(-500, 501, size=(len(x), len(x)))

    return np.max(np.abs(x*A - A))

In [ ]:
print("f1(x):", f1(x))
print("f2(x):", f2(x))
print("f3(x):", f3(x))
print("f4(x):", f4(x))
print("f5(x):", f5(x))

In [20]:
def calculate_min_x(func, no_of_agents, x, es, alpha):
    min_x = x
    for _ in range(es):
        xm = min_x + alpha * np.random.normal(size=len(x))
        xm = check_x(xm)
        if func(xm) < func(min_x):
            min_x = xm
    return min_x

In [21]:
def get_population(number_of_variables, population_size):
    return np.random.normal(size=(population_size, number_of_variables))

In [22]:
initial_alpha = 0.5
dimension = 10
population_size = 5
iterations = 100
functions = [f1, f2, f3, f4, f5]
population = get_population(dimension, population_size)

# Lab2

In [33]:
def population_v2_adaptive(population, function, dimension, max_NFE = 1000, initial_alpha = 0.5,):
    NFE = 0
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for agent in population:
            new_agents = [agent + alpha * np.random.normal(size=dimension) for _ in range(5)]
            best_agent = min(new_agents, key=function)
            new_population.append(best_agent)

            NFE += dimension

            if NFE >= max_NFE:
                break

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

        if NFE >= max_NFE:
            break

    return population

In [34]:
print("f1(x):", population_v2_adaptive(population, f1, dimension))
print("f2(x):", population_v2_adaptive(population, f2, dimension))
print("f3(x):", population_v2_adaptive(population, f3, dimension))
print("f4(x):", population_v2_adaptive(population, f4, dimension))
print("f5(x):", population_v2_adaptive(population, f5, dimension))

f1(x): [array([ 0.11750775, -0.44404495, -0.21848158, -0.40152405,  0.08502493,
        0.33030404, -0.48885297,  0.65113239,  0.71675038,  0.27460054]), array([ 0.99048162,  1.26568848, -0.38449369,  1.31969432, -0.35320512,
       -0.74774516, -0.22734794,  0.04157135, -0.19442679,  0.11893802]), array([-0.6596545 , -0.84798283,  0.15309119,  0.39784247, -0.37094897,
       -0.10082373,  0.25277788,  0.25331901, -0.19290092,  0.26579353]), array([-0.41696145,  0.28593243,  0.14565162, -1.01382789,  0.42244268,
       -0.82137078,  0.99864761, -1.41629399,  0.80002133,  0.35329819]), array([ 0.11978443, -0.08211577, -0.60943137,  0.05156146, -0.27982093,
        0.30985103,  0.0998803 ,  0.19906729, -0.06598721, -0.20646767])]
f2(x): [array([ 0.47110749, -0.30221489, -0.61419004, -0.56351924,  0.96760253,
       -0.05208055, -0.17976421,  0.58493683,  0.39819158,  0.2955008 ]), array([ 0.9460665 ,  0.67972964, -1.14876697,  2.20706417, -1.05554057,
       -0.8413568 , -1.40159008, -0.

# Lab3

In [49]:
def pc_function(iter_no):
    return max(0.5, 1 / (1 + iter_no))

In [50]:
def pm_function(iter_no):
    return max(0.01, 0.1 / (1 + iter_no))

In [38]:
def sphere_function(x):
    return np.sum(np.square(x))

In [53]:
def CGA_adaptiveV1_greedy(population, function, dimension, pc_function = pc_function, pm_function = pm_function, max_NFE = 1000):
    NFE = 0
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        pc = pc_function(iter_no)
        pm = pm_function(iter_no)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, dimension - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population

In [76]:
final_population = CGA_adaptiveV1_greedy(population, f1, dimension)
print("Best solution (CGA):", min(final_population, key=f1))


Best solution (CGA): [-0.09196176 -0.14153996 -1.11921004 -0.3453729  -0.3408875   0.23848204
 -0.13813654  0.12779614  0.09208809 -0.80516765]


# Lab4

In [41]:
def pc_function_fitness(fitness):
    return 0.5 + 0.4 * fitness

In [42]:
def pm_function_fitness(fitness):
    return 0.1 - 0.08 * fitness

In [57]:
def RGA_4(population, function, dimension,  pc=0.9, pm=0.1, max_NFE=1000):
    NFE = 0
    while NFE < max_NFE:
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = population[np.random.choice(len(population), 2, replace=False)]

            if np.random.rand() < pc:
                alpha = 0.1
                child1 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
                child2 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child1[mutation_idx] += np.random.normal()
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child2[mutation_idx] += np.random.normal()

            child1 = np.clip(child1, -10, 10)
            child2 = np.clip(child2, -10, 10)

            new_population.extend([child1, child2])

        combined_population = np.vstack((population, new_population))
        combined_population = sorted(combined_population, key=function)
        population = np.array(combined_population[:len(population)])

        NFE += len(new_population)

    return population

In [58]:
final_population_v2 = RGA_4(population, f1, dimension)
print("Best solution (RGA_4):", min(final_population_v2, key=f1))

Best solution (RGA_4): [-4.54518989e-02 -1.00433991e-01  1.04255124e-01  1.07370270e-01
  1.40286223e-05  2.12512483e-04 -1.74438506e-01  1.89941376e-01
 -6.78451144e-02 -1.39195671e-01]


# Lab5

In [59]:
def DE_best_1_exp(population, function, dimension, F=0.8, CR=0.9, max_NFE=1000):
    NFE = 0
    population_size = population.shape[0]

    while NFE < max_NFE:
        new_population = []

        for i in range(population_size):
            best = min(population, key=function)
            r1, r2 = population[np.random.choice(population_size, 2, replace=False)]
            mutant = best + F * (r1 - r2)

            trial = np.copy(population[i])
            start_idx = np.random.randint(dimension)
            for j in range(dimension):
                if np.random.rand() < CR or j == start_idx:
                    trial[j] = mutant[j]

            trial = np.clip(trial, -10, 10)

            if function(trial) < function(population[i]):
                new_population.append(trial)
            else:
                new_population.append(population[i])

        population = np.array(new_population)
        NFE += population_size

    return population


In [60]:
final_population_v2 = DE_best_1_exp(population, f1, dimension)
print("Best solution (DE_best_1_exp):", min(final_population_v2, key=f1))

Best solution (DE_best_1_exp): [ 0.00851977 -0.00119695 -1.06069692 -0.00139594 -0.45633523  0.00186699
 -0.12003704  0.00292447  0.0412336  -0.07499174]


## Butnaru Teodor

# Lab2

In [63]:
def population_v1_self_adaptive(population, function, dimension = 10, max_NFE = 1000):
    NFE = 0
    iter_no = 0
    alfa = [[30, -30] for _ in range(len(population))]

    while NFE < max_NFE:
        iter_no += 1
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for i, agent in enumerate(population):
            xm = [
                agent[0] + alfa[i][0] * random.uniform(-1, 1),
                agent[1] + alfa[i][1] * random.uniform(-1, 1)
            ]

            if function(agent) > function(xm):
                new_population.append(xm)
            else:
                new_population.append(agent)

            alfa[i][0] += alfa[i][0] * random.uniform(-1, 1)
            alfa[i][1] += alfa[i][1] * random.uniform(-1, 1)

            NFE += len(population)

            if NFE >= max_NFE:
                break

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

        if NFE >= max_NFE:
            break

    return population

In [64]:
print("f1(x):", population_v1_self_adaptive(population, f1))
print("f2(x):", population_v1_self_adaptive(population, f2))
print("f3(x):", population_v1_self_adaptive(population, f3))
print("f4(x):", population_v1_self_adaptive(population, f4))
print("f5(x):", population_v1_self_adaptive(population, f5))

f1(x): [[-0.01352118905264554, 0.23780518367551345], [1.3499336578485661, -0.08951379346853697], [0.6059359605029607, -1.8878063190396355], [-0.0019102071899059665, -0.36695316886421553], [0.40390723859055205, -0.14028701763243034]]
f2(x): [[0.1643909902207465, 625.3907170406704], [-0.0021615363966688535, -47.422642471703924], [0.4891805213640749, 13.441851391764589], [0.003972945903671671, -11.448321702393997], [-0.40452128573157986, 1.377357167094153]]
f3(x): [[-28.359707369118542, -11.097171522078357], [1.014585836879162, -0.004974758735202128], [-27.26861084201984, -0.00017030780080262101], [-131.58996517223338, 0.019983778483975482], [-1.0840844759450892, -0.0423792542690081]]
f4(x): [[-1.0004525080330997, -1.9835426620082488], [0.045324683325663306, -0.6514043361051425], [-2.2945266372434263, 8.44921491242443], [0.1998215862532821, -2.472225294786878], [-0.10770754506233315, -16.98436390512473]]
f5(x): [[1.9628397620755038, 0.4526883804446355], [1.3324385151286102, 4.535754945005

# Lab3

In [65]:
def pc_function_fit(avg_fitness):
    return max(0.5, 1 / (1 + avg_fitness))

def pm_function_fit(avg_fitness):
    return min(0.1, 1 / (1 + avg_fitness))

def sphere_function_fit(agent):
    return sum(x**2 for x in agent)

In [66]:
def CGA_adaptiveV2_greedy(population, function ,dimension, pc_function = pc_function_fit, pm_function = pm_function_fit, max_NFE = 1000):
    NFE = 0
    max_NFE = 1000
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1

        # Calculate fitness-based probabilities (pc and pm) based on the population
        avg_fitness = np.mean([function(agent) for agent in population])

        # Adjust pc and pm based on average fitness
        pc = pc_function_fit(avg_fitness)
        pm = pm_function_fit(avg_fitness)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            # Crossover
            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, dimension - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            # Mutation
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        # Sort the new population based on fitness (function value)
        new_population.sort(key=function)

        # Preserve the best solution (elitism)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent  # Replace worst solution with best solution

        # Update population
        population = new_population

        # Update NFE count (each agent's fitness is evaluated once)
        NFE += len(new_population)

    return population

In [69]:
final_population = CGA_adaptiveV2_greedy(population, f1, dimension, pc_function, pm_function)
print("Best solution (CGA):", min(final_population, key=sphere_function))

Best solution (CGA): [ 0.01573799  0.10432283 -0.06915167  0.03161557  0.7806975  -0.21944339
 -0.00609168  0.19366724  0.09208809 -0.00288048]


# Lab4

In [77]:
def RGA_3(population, function, dimension, pc = 0.8, pm = 0.1, max_NFE = 1000):
    NFE = 0

    while NFE < max_NFE:

        fitness_values = np.array([function(individual) for individual in population])
        NFE += len(population)

        new_population = []

        for _ in range(len(population) // 2):

            parent1 = population[np.random.randint(len(population))]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                d = np.abs(parent1 - parent2)
                lower = np.minimum(parent1, parent2) - 0.5 * d
                upper = np.maximum(parent1, parent2) + 0.5 * d
                child1 = np.random.uniform(lower, upper)
                child2 = np.random.uniform(lower, upper)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child1[mutation_idx] += np.random.normal()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child2[mutation_idx] += np.random.normal()

            new_population.extend([child1, child2])

        new_population.sort(key=function)
        population = new_population[:len(population)]

    return population

In [79]:
print(RGA_3(population, f1, dimension))

[array([-0.95322677,  0.82047725,  3.83667347,  0.21932689,  0.7458989 ,
        4.32276803, -2.09161965, -3.17407112,  3.66684596, -1.06756471]), array([-0.95322677,  0.98212245,  3.83667347,  0.21932689,  0.7458989 ,
        4.32276803, -2.20742829, -3.17407112,  3.66684596, -1.06756471]), array([-0.95322677,  0.98212245,  3.83667347,  0.21932689,  0.7458989 ,
        4.32276803, -2.20742829, -3.17407112,  3.66684596, -1.06756471]), array([-0.95322677,  2.53321264,  3.83667347,  0.21932689,  0.7458989 ,
        4.32276803, -2.48828701, -3.17407112,  3.66684596, -1.06756471])]


In [72]:
best_solution, best_fitness, nfe_used = RGA_3(
    pc=0.8,
    pm=0.1,
    population=population,
    function=f1,
    dimension=dimension
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

Best Solution: [-0.09786387 -2.31174493 -4.20693372  0.49815267 -2.76448368  0.04635817
 -1.17218954  0.60724168  1.53723117  1.75383046]
Best Fitness: 38.126480225321
Number of Function Evaluations: 1001


# Lab5

In [82]:
def DE_rand_to_best_1_exp(population, function, dimension, F = 0.5, CR = 0.9,  max_NFE = 1000):
    NFE = 0
    population_size = len(population)

    while NFE < max_NFE:
        fitness_values = np.array([function(ind) for ind in population])
        best_idx = np.argmin(fitness_values)
        best_individual = population[best_idx]
        NFE += len(population)

        new_population = []

        for i in range(population_size):
            x_rand = population[np.random.randint(population_size)]
            x1, x2 = population[np.random.choice([j for j in range(population_size) if j != i], size=2, replace=False)]
            mutant_vector = x_rand + F * (best_individual - x_rand) + F * (x1 - x2)

            trial_vector = population[i].copy()
            start_index = np.random.randint(dimension)
            length = 0
            while np.random.rand() < CR and length < dimension:
                trial_vector[(start_index + length) % dimension] = mutant_vector[(start_index + length) % dimension]
                length += 1

            if function(trial_vector) < fitness_values[i]:
                new_population.append(trial_vector)
            else:
                new_population.append(population[i])

        population = np.array(new_population)

    return population


In [83]:
print(DE_rand_to_best_1_exp(population, f1, dimension))

[[-0.14049641 -0.57478913 -0.68415739 -0.18784577 -0.42693908  0.18888711
  -0.00232917 -0.0323413   0.24574033  0.14048701]
 [-0.14049641 -0.57478913 -0.68415739 -0.18784577 -0.42693908  0.18888711
  -0.00232917 -0.0323413   0.24574033  0.14048701]
 [-0.14049641 -0.57478913 -0.68415739 -0.18784577 -0.42693908  0.18888711
  -0.00232917 -0.0323413   0.24574033  0.14048701]
 [-0.14049641 -0.57478913 -0.68415739 -0.18784577 -0.42693908  0.18888711
  -0.00232917 -0.0323413   0.24574033  0.14048701]
 [-0.14049641 -0.57478913 -0.68415739 -0.18784577 -0.42693908  0.18888711
  -0.00232917 -0.0323413   0.24574033  0.14048701]]


In [74]:
best_solution, best_fitness, nfe_used = DE_rand_to_best_1_exp(
    F=0.5,
    CR=0.9,
    population=population,
    function=f2,
    dimension=dimension
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

Best Solution: [ 0.63813162 -0.22545519 -0.758855   -0.54907042  0.80964292  0.09009673
 -0.04221031 -0.15259839  0.38692995  0.18542703]
Best Fitness: 1.5824726731828715
Number of Function Evaluations: 1000


**Laboratory 6. Testing meta-heuristics**

In [84]:
import pandas as pd

In [86]:
D = [5, 10]
NEF = 1000

functions = [f1, f2, f3, f4, f5]
population_size = 5
algoritms = [population_v2_adaptive, CGA_adaptiveV1_greedy, RGA_4, DE_best_1_exp, population_v1_self_adaptive, CGA_adaptiveV2_greedy, RGA_3, DE_rand_to_best_1_exp]


for dimension in D:
  print("Dimension: ", dimension)
  population = get_population(dimension, population_size)
  for algoritm in algoritms:
    print("Algoritm: ", algoritm.__name__)
    for function in functions:
      print("Function: ", function.__name__)
      results = []
      for i in range(10):
        population = get_population(dimension, population_size)
        result_populations = algoritm(population, function, dimension, max_NFE=NEF)
        results.append(function(min(result_populations, key=function)))

      df = pd.DataFrame({"results" : results})
      stats = df.describe().loc[['min', 'max', 'mean', 'std']]
      print(stats)
      print()




Dimension:  5
Algoritm:  population_v2_adaptive
Function:  f1
       results
min   0.000402
max   0.145336
mean  0.016914
std   0.045311

Function:  f2
       results
min   0.000125
max   0.030256
mean  0.006509
std   0.010377

Function:  f3
         results
min     0.579690
max   128.393027
mean   18.134204
std    39.318096

Function:  f4
       results
min   0.000832
max   1.503928
mean  0.456681
std   0.484887

Function:  f5
         results
min   254.049655
max   930.880882
mean  532.707284
std   226.605983

Algoritm:  CGA_adaptiveV1_greedy
Function:  f1
       results
min   0.146684
max   1.525849
mean  0.891718
std   0.513021

Function:  f2
       results
min   0.061483
max   1.827931
mean  0.583181
std   0.504534

Function:  f3
           results
min      61.362127
max   14079.761059
mean   2636.621743
std    4231.252033

Function:  f4
       results
min   0.178737
max   2.210423
mean  1.149990
std   0.593612

Function:  f5
          results
min    118.206477
max   1017.636608
m